In [1]:
import pandas as pd
import numpy as np

In [2]:
ten_day_df = pd.read_csv("callups_10day_tidy.csv")
two_way_df = pd.read_csv("two_way_contracts.csv")
conversions_df = pd.read_csv("two_way_conversions.csv")
sportsradar_df = pd.read_csv("../raw/gleague_player_season_stats_2024_REG.csv")

ten_day_df.head(), two_way_df.head(), conversions_df.head()

(         date   season         player_name               nba_team  \
 0  2007-01-06  2006-07     Thompson, Dijon          Atlanta Hawks   
 1  2007-01-27  2006-07  Richardson, Jeremy          Atlanta Hawks   
 2  2007-04-04  2006-07     Pinkney, Kevinn         Boston Celtics   
 3  2007-04-02  2006-07       Willis, Kevin       Dallas Mavericks   
 4  2007-01-17  2006-07      Major, Renaldo  Golden State Warriors   
 
                                        contract_type  
 0      Two 10-day contracts (only 1st date is shown)  
 1      Two 10-day contracts (only 1st date is shown)  
 2                                    10-day contract  
 3  10-day contract followed by signing for the re...  
 4                                    10-day contract  ,
       player_name        date        nba_team   season    contract_type
 0      Eli Ndiaye  2025-07-01   atlanta hawks  2025-26  two-way signing
 1    Jacob Toppin  2025-07-01   atlanta hawks  2025-26  two-way signing
 2   Caleb Houston  20

In [3]:
def clean_name(name):
    if isinstance(name, str) and "," in name:
        last, first = name.split(",", 1)
        return first.strip() + " " + last.strip()
    return name.strip() if isinstance(name, str) else name

# Clean name fields
ten_day_df["player_name"] = ten_day_df["player_name"].apply(clean_name)
two_way_df["player_name"] = two_way_df["player_name"].apply(clean_name)
conversions_df["player_name"] = conversions_df["player_name"].apply(clean_name)

# SportsRadar originally uses "full_name"
sportsradar_df = sportsradar_df.rename(columns={"full_name": "player_name"})
sportsradar_df["player_name"] = sportsradar_df["player_name"].apply(clean_name)

In [4]:
contracts_df = pd.concat([ten_day_df, two_way_df, conversions_df], ignore_index=True)
contracts_df.head()


,date,season,player_name,nba_team,contract_type
0,2007-01-06,2006-07,Dijon Thompson,Atlanta Hawks,Two 10-day contracts (only 1st date is shown)
1,2007-01-27,2006-07,Jeremy Richardson,Atlanta Hawks,Two 10-day contracts (only 1st date is shown)
2,2007-04-04,2006-07,Kevinn Pinkney,Boston Celtics,10-day contract
3,2007-04-02,2006-07,Kevin Willis,Dallas Mavericks,10-day contract followed by signing for the re...
4,2007-01-17,2006-07,Renaldo Major,Golden State Warriors,10-day contract


In [5]:
final_df = contracts_df.merge(sportsradar_df,on="player_name",how="left",)

In [6]:
final_df.to_csv("combined_gleague_contracts.csv", index=False)

In [7]:
df = pd.read_csv("combined_gleague_contracts.csv")
sorted_df = df.sort_values("season", ascending=False)
sorted_df.to_csv("combined_gleague_contracts.csv", index=False)
sorted_df

,date,season,player_name,nba_team,contract_type,season_id,season_type,team_id,player_id,position,...,rebounds,steals,blocks,turnovers,fgm,fga,tpm,tpa,ftm,fta
865,2025-07-01,2025-26,Kobe Sanders,los angeles clippers,two-way signing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
848,2025-07-01,2025-26,Moussa Cisse,dallas mavericks,two-way signing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
850,2025-07-01,2025-26,Spencer Jones,denver nuggets,two-way signing,2024.0,REG,bbeebcb5-0d9b-4992-bcba-cbde3ec60628,27403427-4b94-4336-8b1a-6c6035b7be1c,F,...,87.0,18.0,17.0,15.0,72.0,131.0,36.0,72.0,6.0,8.0
851,2025-07-01,2025-26,Curtis Jones,denver nuggets,two-way signing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
852,2025-07-01,2025-26,Tolu Smith,detroit pistons,two-way signing,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16,2007-01-05,2006-07,Andre Brown,Seattle SuperSonics,Two 10-day contracts followed by signing for t...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,2007-03-25,2006-07,Luke Jackson,Toronto Raptors,Two 10-day contracts followed by signing for t...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,2007-02-05,2006-07,Louis Amundson,Utah Jazz,Two 10-day contracts (only 1st date is shown),NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,2007-02-28,2006-07,Mike Hall,Washington Wizards,Two 10-day contracts followed by signing for t...,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
